##### import libraries 

In [1]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms
from torch.autograd import Variable
import fashion_mnist

##### Hyper Params

In [2]:
# Hyper Params 
input_size = 784 
hidden_size = 500 
num_classes = 10 
num_epoch  = 10
batch_size = 100
learning_rate  = 0.0005

In [3]:
# Load fashion mnist data 
train_dataset =fashion_mnist.MNIST(root='fashion_mnist/', train=True, transform=transforms.ToTensor(), download=True)

In [4]:
test_dataset = dsets.MNIST(root='fashion_mnist/', 
                           train=False, 
                           transform=transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size = batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False
                                         )

In [6]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out 
    

In [7]:
net = Net(input_size, hidden_size, num_classes)

In [8]:
net.cuda()

Net (
  (fc1): Linear (784 -> 500)
  (relu): ReLU ()
  (fc2): Linear (500 -> 10)
)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

In [10]:
best_loss = 10
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        # convert torch tensor to Variable 
        images = Variable(images.view(-1, 28 * 28 ).cuda())
        labels = Variable(labels.cuda())
        
        # forward 
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1)%100 == 0:
            print('Epoch [%d/%d], Step[%d/%d], Loss:%.4f' % (epoch + 1, num_epoch + 1, i+1, len(train_dataset)
                                                            //batch_size, loss.data[0]))
        if best_loss > loss.data[0]:
            print("best_loss => ", best_loss, "loss current => ", loss.data[0])
            best_loss = loss.data[0]
            torch.save(net, 'model.pkl')
            print('Epoch [%d/%d], Step[%d/%d], Loss:%.4f' % (epoch + 1, num_epoch + 1, i+1, len(train_dataset)
                                                //batch_size, loss.data[0]))
        

best_loss =>  10 loss current =>  2.3296163082122803
Epoch [1/11], Step[1/600], Loss:2.3296
best_loss =>  2.3296163082122803 loss current =>  2.2403178215026855
Epoch [1/11], Step[2/600], Loss:2.2403
best_loss =>  2.2403178215026855 loss current =>  2.134056806564331
Epoch [1/11], Step[3/600], Loss:2.1341
best_loss =>  2.134056806564331 loss current =>  2.0588386058807373
Epoch [1/11], Step[4/600], Loss:2.0588
best_loss =>  2.0588386058807373 loss current =>  2.0086233615875244
Epoch [1/11], Step[5/600], Loss:2.0086
best_loss =>  2.0086233615875244 loss current =>  1.9413663148880005
Epoch [1/11], Step[6/600], Loss:1.9414
best_loss =>  1.9413663148880005 loss current =>  1.872633934020996
Epoch [1/11], Step[7/600], Loss:1.8726
best_loss =>  1.872633934020996 loss current =>  1.6484547853469849


/home/rajat/torch/lib/python3.5/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch [1/11], Step[8/600], Loss:1.6485
best_loss =>  1.6484547853469849 loss current =>  1.6455632448196411
Epoch [1/11], Step[10/600], Loss:1.6456
best_loss =>  1.6455632448196411 loss current =>  1.5328364372253418
Epoch [1/11], Step[11/600], Loss:1.5328
best_loss =>  1.5328364372253418 loss current =>  1.5165317058563232
Epoch [1/11], Step[13/600], Loss:1.5165
best_loss =>  1.5165317058563232 loss current =>  1.3718931674957275
Epoch [1/11], Step[14/600], Loss:1.3719
best_loss =>  1.3718931674957275 loss current =>  1.3094775676727295
Epoch [1/11], Step[15/600], Loss:1.3095
best_loss =>  1.3094775676727295 loss current =>  1.2296804189682007
Epoch [1/11], Step[17/600], Loss:1.2297
best_loss =>  1.2296804189682007 loss current =>  1.1447484493255615
Epoch [1/11], Step[19/600], Loss:1.1447
best_loss =>  1.1447484493255615 loss current =>  1.0274817943572998
Epoch [1/11], Step[22/600], Loss:1.0275
best_loss =>  1.0274817943572998 loss current =>  0.915382981300354
Epoch [1/11], Step[26

In [11]:
# Test the Model
correct = 0
total = 0
#net_new = torch.load('model.pkl')

# initalise model class 
# net_new = Net(*args, **kwargs)
# net_new.load_state_dict(torch.load('model.pkl'))
net_new = torch.load('model.pkl')
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28)).cuda()
    outputs = net_new(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
# Save the Model
# torch.save(net.state_dict(), 'model.pkl')

Accuracy of the network on the 10000 test images: 88 %


In [12]:
net_new

Net (
  (fc1): Linear (784 -> 500)
  (relu): ReLU ()
  (fc2): Linear (500 -> 10)
)

In [13]:
net.fc1

Linear (784 -> 500)